# Esame Ferrara, 22/02/2023

## Prima domanda: Illustrare lo Small World model

Lo Small World è uno dei quattro newtwork fondamentali che abbiamo studiato, spesso ci si riferisce ad esso con il nome di "Watts-Strogatz model", dai suoi creatori.
L'idea alla base dello Small World è che si parte da un k-regular network con k strettamente pari, questo non è altro che un network in cui ogni nodo ha degree pari a k, ad esempio un 3-regular network è un network in cui ogni nodo è connesso ad altri 3 nodi.
Chiaramente la degree distribution di un simile network è una delta dirac sul valore k.
Una volta che si ha il k-regular network si può applicare un processo detto "rewiring": Si può immaginare il network come un anello di nodi, si itera su ogni nodo in senso orario (per esempio), e si scrivono i link che ha il nodo i-esimo nel suo verso orario, per ognuno di questi link si estrarrà un numero random float compreso fra 0 e 1, e si comparerà con la cosidettà "rewiring probability" p, se il numero estratto è più basso di p il link viene spezzato, e si riconnetterà il nodo con un nodo "lontano", dove "lontano" significa che non era connesso al nodo originale.
Una volta fatto ciò si ottiene uno Small World network.
Chiaramente il risultato finale dipende dalla rewiring probability p, se p = 1 il rewiring è totale e non resta nessuna informazione del network k-regular originale (a parte chiaramente il degree medio che resta k e il numero di nodi N), si ottiene un grafico che è a tutti gli effetti una realizzazione di Erdos-Reinyi equivalente ad uno generato - considerando che il k medio resta quello ereditato dal k-regular - con la p (di ER) che è pari generalmente a k/N, dove N è il numero di nodi del network, ereditato chiaramente anch'esso dal k-regular, e k è proprio quello del k-regular.
Per rewiring probability più basse abbiamo uno Small World classico, le sue caratteristiche essenziali sono che l'average shortest path lenght è generalmente più basso di un grafico che ad esempio otterremmo applicando un configuration model e ricalcolandolo, da ciò la classica frase "it's a small world!" per indicare che l'ASPL fra due nodi è in effetti più basso di un grafico come ER, dove tende a infinito per N che tende a infinito.
Si può dire la stessa cosa del Global Clustering Coefficient, questo risulta mediamente più alto di quello che otterremo applicando il configuration model, al contrario di ER dove il CC tende a 0 per N tendente a infinito (si può dimostrare che il CC in un ER va come p, dove con p=1 chiaramente tutti i triangoli sono chiusi, quindi è pari a 1).
In generale la degree distribution di uno Small World è pari ad una gaussiana, infatti il k medio si allarga (considerando che si partiva da una delta dirac!), ha quindi un decadimento esponenziale, più simile quindi alla coda di un ER che ad un grafico scale-free come Barabasi_Albert o Borgatti_Everett.

## Seconda domanda: Illustrare il concetto di percolazione e perché è rilevante in Complex Networks

Per spiegare la percolazione possiamo immaginare uno strato di roccia bidimenzionale che ha dell'acqua sopra ed una cavità sotto.
Se rimuoviamo cerchi di area sempre uguali nella roccia, l'acqua prima o poi riuscirà a passare.
Si può scrivere una quantità r chiamata "order parameter" che è proprio il rapporto fra il volume (o l'area, in 2D) che è stato "forato" e quello totale.
Possiamo immaginare un lattice bidimensionale, consideriamo quindi che ogni nodo in questo lattice ha una probabilità p di essere presente o no, quale è la probabilità minima necessaria affinché si può arrivare da un estremo all'altro del lattice?
In un lattice 2D questa p_c è pari a circa 0.59.
Nello studio dei Complex Network la percolazione è un concetto che ha molte applicazioni, ad esempio con un approccio basato sulla percolazione possiamo chiederci quale sia la probabilità necessaria associata alla presenza di nodi per cui abbiamo una giant component (e siamo quindi in grado, generalmente, di andare da nodo A a nodo B per qualsiasi A e B), utilizzando questo tipo di approccio si trova che la p_c è pari a (<k^2> - 2<k>) / (<k^2> - <k>), ciò deriva dalla condizione di Molloy_Reed in cui <k^2> - 2<k> > 0 , in particolare si può anche indagare sulla vulnerabilità dei network, ad esempio ci si può chiedere: se si rimuovono nodi casualmente con probabilità p, quale è il valore che p deve avere per riuscire a "rompere" la giant component? Per un network Scale Free tipo Barabasi Albert viene fuori che p_c è quasi 1, ovvero è praticamente indistruttibile contro attachi random di questo tipo.
In effetti per rompere il network sarebbe meglio utilizzare attacchi più strategici, ad esempio attaccando i nodi con alta betweenness.
Un approccio legato alla percolazione è anche importante in un network dei contatti che sta alla base di un modello SIR per esempio, la percolazione critica a quel punto è la probabilità che dovremmo usare nel rimuovere casualmente ogni nodo per "interrompere" lo spreading di un'epidemia, ovvero può considerarsi ad esempio legata alla frazione di popolazione che bisogna vaccinare.

## Terza domanda: codice!

In [ ]:
#importiamo qualche utile libreria!
import numpy as np
import igraph as ig
import random
import matplotlib.pyplot as plt

In [ ]:
#Fortunatamente ho già l'edgelist del network degli aereoporti sul pc, chiaramente non pesata e non diretta.

g = ig.Graph.Read_Ncol("airport_original_edgelist.txt", names=True, directed=False) #carichiamo il grafico dalla edgelist!

In [ ]:
ig.plot(g, layout="graphopt", vertex_size=5, bbox=(800, 450)) #plottiamo il network

In [ ]:
# Prendiamoci la giant component:
airports_original = g.connected_components().subgraph(0) 

In [ ]:
ig.plot(airports_original, layout="graphopt", vertex_size=5, bbox=(800, 450))

In [ ]:
#Scriviamo un codice che ci permetta di fare il configuration model:

In [ ]:
def edge_switcher(edgelist):
    # mi genero un vettore "random order", con l'idea che fa da indice (ogni indice NON si ripete mai) ma in ordine randomico!
    random_order = [i for i in range(len(edgelist))]
    random.shuffle(random_order)
    
    for i in range(len(edgelist)):
        #mi genero un numero int random casuale che chiaramente farà da indice quindi è compreso fra 0 e len(edgelist)-1
        rand = random.randint(0,len(edgelist)-1)
        
        #mi trovo un link casuale, ad esempio edgelist[randomorder[i]], e li richiamo in modo più leggibile
        A_1 = edgelist[random_order[i]][0]
        B_1 = edgelist[random_order[i]][1]
        
        #faccio la stessa cosa però con un link nella posizione rand
        A_2 = edgelist[rand][0]
        B_2 = edgelist[rand][1]
        
        
        # questi if fanno da controllo, essenzialmente controllano che i link una volta switchati non siano self loops o multi loops, ma invece di andare avanti nel codice e basta, abbasso l'indice i di uno e uscendo dal for posso riprovare a fare lo switch finchè non funziona a dovere (ovvero no self-loops no multi-links), la funzione max serve a fare in modo che se lo switch fallisce proprio all'iterazione 0 non mi ritrovo un indice i negativo!
        if A_1 == B_2:
            i = max(i-1,0)
            continue

        if B_1 == A_2 :
            i = max(i-1,0)
            continue

        if (A_1,B_2) in edgelist or (A_2,B_1) in edgelist or (B_2,A_1) in edgelist or (B_1,A_2) in edgelist:
            i = max(i-1,0)
            continue
        
        
        #se tutto funziona a dovere, switcho in effetto i link (ovvero se ho A-B e C-D alla fine avrò A-D e C-B)
        tuple1 = (A_1, B_2)
        tuple2 = (A_2, B_1)
        edgelist[random_order[i]] = tuple1
        edgelist[rand] = tuple2
        
        
    return edgelist


In [ ]:
# per applicare il configuration model ci serve l'edgelist del network, prendiamocela

edgelist = airports_original.get_edgelist() #chiaramente prendiamo l'edgelist della giant component

In [ ]:
# adesso scriviamo un ciclo for che ci realizza n realizzazioni del configuration model!

nr = 10 # vogliamo 100 realizzazioni
random_edgelist_vector = [0] * nr # inizializziamo la lista lunga nr

for i in range(nr):
    random_edgelist_vector[i] = edge_switcher(edgelist)
    
# il ciclo for sopra potrebbe metterci un po' ad eseguire, il tempo scala linearmente con nr, per nr = 1 ci mette circa 6 secondi, quindi con nr = 100 ci metterà circa 10 minuti sul mio portatile

In [ ]:
# guardiamo un network a caso di quelli generati

In [ ]:
index = 6 # cambiare questo indice per vedere grafici diversi!

g = ig.Graph(random_edgelist_vector[index], directed = False)
ig.plot(g, layout="graphopt", vertex_size=5, bbox=(800, 450))

In [ ]:
# adesso calcoliamoci una partition del network originale usando la modularity

community_original = airports_original.community_multilevel() # questo algoritmo dovrebbe basarsi sull'argoritmo di Blondel che massimizza la modularità

In [ ]:
# plottiamo la partizione del network originale
ig.plot(community_original, layout="graphopt", vertex_size=5, bbox=(800, 450))

In [ ]:
# creiamo un for loop per caricare su igraph ognuna di queste partizioni

airports_switched_vector = [0] * nr

for i in range(nr):
    airports_switched_vector[i] = ig.Graph(random_edgelist_vector[i], directed = False) #è una lista di grafici igraph!

In [ ]:
# creiamo un for loop per calcolarci le 100 partitions dei 100 networks utilizzando lo stesso algoritmo

community_switched_vector = [0] * nr

for i in range(nr):
    community_switched_vector[i] = airports_switched_vector[i].community_multilevel()

In [ ]:
# plottiamo la partizione di un network del configuration model a caso
index = 7 # cambiare qui per vedere partizioni diverse!

ig.plot(community_switched_vector[index], layout="graphopt", vertex_size=5, bbox=(800, 450))

In [ ]:
# per testare il codice, calcoliamoci l'ARI fra la partizione originale del network e... se stessa

ARI = ig.compare_communities(community_original, community_original, method="adjusted_rand")
print(ARI) #dovrebbe fare 1

In [ ]:
# adesso calcoliamo l'ARI fra le partizioni trovate e quello originale, chiaramente avremo un vettore di ARI

ARI_vector = [0] * nr

for i in range(nr):
    ARI_vector[i] = ig.compare_communities(community_original, community_switched_vector[i], method="adjusted_rand") #compute the ARI between 2 communities

In [ ]:
ARI_np = np.array(ARI_vector) #first use of numpy in the code!!!

ARI_mean = np.mean(ARI_np)
ARI_std = np.std(ARI_np)

In [ ]:
print("L'ARI medio risulta: {}. La sua deviazione standard è pari a: {}".format(ARI_mean, ARI_std))

Chiaramente l'ARI medio è praticamente nullo, considerando che le partizioni effettuate sulle varie realizzazioni del configuration model non c'entrano niente con quella originale.

L'RI atteso, anche scritto E[RI] è il valore medio atteso dell'RI (Rand Index) trovato su N molto grande di realizzazioni di partizioni randomiche (ad esempio usando il configuration model, proprio come fatto qui). L'ARI in generale si definisce come (RI-E[RI]) / (max(RI) - E[RI]), e si può dimostrare come l'ARI si può poi scrivere in funzione di a,b,c,d che sono, rispettivamente (frazioni di nodi nelle stesse community in part1 e part2, frazioni di nodi in stessa community in part1 ma non in part2, frazioni di nodi nella stessa community in part2 ma non in part1, e frazioni di nodi che non sono nella stessa community sia in part1 che part2).

L'E[ARI] che abbiamo calcolato in questo caso, è supposto che faccia 0, serve come "baseline" per valutare l'ARI ottenuto invece fra due partizioni differenti ma non casuali